In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import pickle
from typing import Union, List
import pandas as pd
from MyDataSet import MultiMaskTimeSeriesDataset

def graph_counter(adj):
    n=len(adj)
    visited=[0]*n
    
    num_of_graph=0
    count=0
    while 1:
        # stack=[0]
        flag=0
        for i in range(n):
            if visited[i]==0:
                stack=[i]
                visited[i]=1
                flag=1
                break
        
        if flag==0:
            break    
        
        count = 1
        while stack:
            node=stack.pop()
            for i in range(n):
                if adj[node][i]==1 and visited[i]==0:
                    visited[i]=1
                    stack.append(i)
                    count+=1
        num_of_graph+=1
        print('num of nodes:',count)
        
    return num_of_graph
def get_adjacency_matrix(distance_df_filename: str, num_of_vertices: int, id_filename: str = None) -> tuple:
    """Generate adjacency matrix.

    Args:
        distance_df_filename (str): path of the csv file contains edges information
        num_of_vertices (int): number of vertices
        id_filename (str, optional): id filename. Defaults to None.

    Returns:
        tuple: two adjacency matrix.
            np.array: connectivity-based adjacency matrix A (A[i, j]=0 or A[i, j]=1)
            np.array: distance-based adjacency matrix A
    """

    if "npy" in distance_df_filename:
        adj_mx = np.load(distance_df_filename)
        return adj_mx, None
    else:
        adjacency_matrix_connectivity = np.zeros((int(num_of_vertices), int(
            num_of_vertices)), dtype=np.float32)
        adjacency_matrix_distance = np.zeros((int(num_of_vertices), int(num_of_vertices)),
                                             dtype=np.float32)
        if id_filename:
            # the id in the distance file does not start from 0, so it needs to be remapped
            with open(id_filename, "r") as f:
                id_dict = {int(i): idx for idx, i in enumerate(
                    f.read().strip().split("\n"))}  # map node idx to 0-based index (start from 0)
            with open(distance_df_filename, "r") as f:
                f.readline()  # omit the first line
                reader = csv.reader(f)
                for row in reader:
                    if len(row) != 3:
                        continue
                    i, j, distance = int(row[0]), int(row[1]), float(row[2])
                    adjacency_matrix_connectivity[id_dict[i], id_dict[j]] = 1
                    adjacency_matrix_connectivity[id_dict[j], id_dict[i]] = 1
                    adjacency_matrix_distance[id_dict[i],
                                              id_dict[j]] = distance
                    adjacency_matrix_distance[id_dict[j],
                                              id_dict[i]] = distance
            return adjacency_matrix_connectivity, adjacency_matrix_distance
        else:
            data=pd.read_csv(distance_df_filename)

            for index, row in data.iterrows():
                i, j, distance = int(row['from']), int(row['to']), float(row['cost'])
                adjacency_matrix_connectivity[i, j] = 1
                adjacency_matrix_connectivity[j, i] = 1
                adjacency_matrix_distance[i, j] = distance
                adjacency_matrix_distance[j, i] = distance
            return adjacency_matrix_connectivity, adjacency_matrix_distance
def time_unit_to_nanoseconds(time_unit: str):
    # check_time_unit(time_unit)
    if time_unit == 'year':
        return 365.2425 * 24 * 60 * 60 * 10**9
    elif time_unit == 'week':
        time_unit = 'W'
    return pd.Timedelta('1' + time_unit).value

def datetime_encoded(dataset : pd.DataFrame, units: Union[str, List]) -> pd.DataFrame:
    r"""Transform dataset's temporal index into covariates using sinusoidal
    transformations. Each temporal unit is used as period to compute the
    operations, obtaining two feature (:math:`\sin` and :math:`\cos`) for
    each unit."""

    datetime = dict()
    for unit in units:
        nano_unit = time_unit_to_nanoseconds(unit)
        nano_sec =  dataset[unit]* (2 * np.pi / nano_unit)
        datetime[unit + '_sin'] = np.sin(nano_sec)
        datetime[unit + '_cos'] = np.cos(nano_sec)
    return pd.DataFrame(datetime, dtype=np.float32)

df = pd.DataFrame({'time': pd.date_range('2020-01-01', periods=288, freq='5min')})
df['day']=df['time'].dt.time
df['day']=df['day'].apply(lambda x: (x.hour*3600+x.minute*60+x.second)*10**9)
dateencoded=datetime_encoded(df, ['day'])
print(dateencoded.shape)

file_path=r'D:\WorkPath\Models\ImputeFormer\Data\raw_data\METR-LA\node_values.npy'

data=np.load(file_path)
num_of_vertices = data.shape[1]
data.shape

In [ ]:
import sys
from turtle import distance
sys.path

adj_path=r'D:\WorkPath\Models\ImputeFormer\Data\raw_data\METR-LA\adj_mat.npy'
adj,distance= get_adjacency_matrix(adj_path,num_of_vertices,None)

graph_counter(adj)

In [ ]:
k={}
for head_ in range(0,207):
    road=[head_]
    head=head_
    for i in range(100):
        for j in range(num_of_vertices):
            if adj[head][j] > 0.1 and j not in road :
                road.append(j)
                head = j
                break
    # print(len(road))
    k[head_]=len(road)    
    # print(road)

In [ ]:
road=[102]
head=102
for i in range(100):
    for j in range(num_of_vertices):
        if adj[head][j] > 0.01 and j not in road :
            road.append(j)
            head = j
            break
print(len(road))    
print(road)

road = [6 ,91, 15, 33, 93, 136, 17, 12, 80, 128, 94, 74, 72, 69, 47, 52, 73, 70, 98, 96, 97, 127, 154, 155, 157, 159, 160, 161, 162, 163, 187, 188, 198, 206]          

In [ ]:
speed_matrix=data[:12*24*2,road,0]

plt.figure(figsize=(15,6))
plt.imshow(speed_matrix.T,aspect='auto',cmap='jet',origin='lower')
plt.colorbar()
plt.show()

In [33]:
route = '006'
direction = 'N'
route_start = 0
route_len =32
missing_rate = 0.3
missing_type = ['random','linear','block','mixed']
missing_type_ind = 0
num_masks = 40
data_type=['source_train','source_test','target_train','target_test']
data_type_ind = 0

In [ ]:
speed_matrix= data[:,road,0]
hot_map_data=speed_matrix.reshape(-1,288,len(road))
hot_map_data.shape

In [ ]:
mask = hot_map_data>1.0
data = speed_matrix.reshape(-1,288,len(road))
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if mask[i, j, k] == 0:
                if i == 0:
                    hot_map_data[i, j, k] = data[i+ 1, j, k ]
                elif i == data.shape[0] - 1:
                    hot_map_data[i, j, k] = data[i - 1, j, k]
                else:
                    hot_map_data[i, j, k] = (data[i - 1, j, k] + data[i+ 1, j, k ]) / 2
hot_map_data.shape


In [ ]:
train_ratio = 0.8
test_ratio = 0.2
source_train_data = hot_map_data[:int(hot_map_data.shape[0]*train_ratio)]
test_data = hot_map_data[int(hot_map_data.shape[0]*train_ratio):int(hot_map_data.shape[0]*(train_ratio+test_ratio))]
target_train_data = hot_map_data[int(hot_map_data.shape[0]*(train_ratio+test_ratio)):]
print(source_train_data.shape,test_data.shape,target_train_data.shape)

In [37]:
test_dir=r'D:\WorkPath\Models\ImputeFormer\Data\source_test_METRLA'
train_dir=r'D:\WorkPath\Models\ImputeFormer\Data\source_train_METRLA'
# os.makedirs(train_dir)
# os.makedirs(test_dir)

In [40]:
train_ratio = 0.8
test_ratio = 0.2
missing_rate = 0.3
x = list(range( int(hot_map_data.shape[0])  ))
random.shuffle(x)

for missing_type_ind in range(4):
    for route_start in range(0,2,5):
        for time_start in [7,11,15,19]:
            
            save_path = train_dir
            save_path_test = test_dir

            # save_path=os.path.join(r'D:\WorkPath\Models\ImputeFormer\Data',data_type[data_type_ind])+'_METRLA'
            # save_path_test=os.path.join(r'D:\WorkPath\Models\ImputeFormer\Data',data_type[data_type_ind+1])+'_METRLA'


            
            # dataset = MultiMaskTimeSeriesDataset(hot_map_data[ x[0:int(train_ratio* hot_map_data.shape[0]) ],
            #                                     time_start*12:(time_start+4)*12,  route_start:route_start+route_len  ],
            #                                      dateencoded.values[time_start*12:(time_start+4)*12,:],
            #                                     missing_rate, missing_type[missing_type_ind], num_masks)
            # dataset_file = os.path.join(save_path, 
            #                             'METR_LA_speed_{}-{}_{}_{}_{}.pkl'.format
            #                             (route, time_start ,route_start ,int(missing_rate*100) , missing_type[missing_type_ind]))
            # pd.to_pickle(dataset, dataset_file)
            
            dataset = MultiMaskTimeSeriesDataset(hot_map_data[ x[int(train_ratio* hot_map_data.shape[0]): ],
                                                time_start*12:(time_start+4)*12,route_start:route_start+route_len],
                                                 dateencoded.values[time_start*12:(time_start+4)*12,:],
                                                missing_rate, missing_type[missing_type_ind], 5)
            dataset_file = os.path.join(save_path_test, 
                                        'METR_LA_speed_{}-{}_{}_{}_{}.pkl'.format
                                        (route, time_start ,route_start ,int(missing_rate*100) , missing_type[missing_type_ind]))
            pd.to_pickle(dataset, dataset_file)